In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import classification_report, confusion_matrix

In [2]:
train_dir = 'Data/Training'

filepaths = []
labels = []

folds = os.listdir(train_dir)

for fold in folds:
    foldpath =os.path.join(train_dir, fold)
    
    files = os.listdir(foldpath)
    for f in files:
        fpath = os.path.join(foldpath, f)
        
        filepaths.append(fpath)
        labels.append(fold)
train_df = pd.DataFrame(data = {'Filepaths':filepaths,'Labels':labels})
train_df

,Filepaths,Labels
0,Data/Training\glioma\Tr-glTr_0000.jpg,glioma
1,Data/Training\glioma\Tr-glTr_0001.jpg,glioma
2,Data/Training\glioma\Tr-glTr_0002.jpg,glioma
3,Data/Training\glioma\Tr-glTr_0003.jpg,glioma
4,Data/Training\glioma\Tr-glTr_0004.jpg,glioma
...,...,...
5707,Data/Training\pituitary\Tr-pi_1452.jpg,pituitary
5708,Data/Training\pituitary\Tr-pi_1453.jpg,pituitary
5709,Data/Training\pituitary\Tr-pi_1454.jpg,pituitary
5710,Data/Training\pituitary\Tr-pi_1455.jpg,pituitary


In [7]:
test_dir = 'Data/Testing'

filepaths = []
labels = []

folds = os.listdir(test_dir)

for fold in folds:
    foldpath =os.path.join(test_dir, fold)
    
    files = os.listdir(foldpath)
    for f in files:
        fpath = os.path.join(foldpath, f)
        
        filepaths.append(fpath)
        labels.append(fold)
ts_df = pd.DataFrame(data = {'Filepaths':filepaths,'Labels':labels})
ts_df

,Filepaths,Labels
0,Data/Testing\glioma\Te-glTr_0000.jpg,glioma
1,Data/Testing\glioma\Te-glTr_0001.jpg,glioma
2,Data/Testing\glioma\Te-glTr_0002.jpg,glioma
3,Data/Testing\glioma\Te-glTr_0003.jpg,glioma
4,Data/Testing\glioma\Te-glTr_0004.jpg,glioma
...,...,...
1306,Data/Testing\pituitary\Te-pi_0295.jpg,pituitary
1307,Data/Testing\pituitary\Te-pi_0296.jpg,pituitary
1308,Data/Testing\pituitary\Te-pi_0297.jpg,pituitary
1309,Data/Testing\pituitary\Te-pi_0298.jpg,pituitary


In [8]:
valid_df, test_Df = train_test_split(ts_df,test_size=0.5,random_state=42)

In [9]:
gen = ImageDataGenerator()

train_gen = gen.flow_from_dataframe(train_df, x_col='Filepaths', y_col='Labels', target_size=(224, 224),
                                   color_mode='rgb', class_mode='categorical' , batch_size=32)

valid_gen = gen.flow_from_dataframe(valid_df, x_col='Filepaths', y_col='Labels', target_size=(224, 224),
                                   color_mode='rgb', class_mode='categorical' , batch_size=32)

test_gen = gen.flow_from_dataframe(test_Df, x_col='Filepaths', y_col='Labels', target_size=(224, 224),
                                   color_mode='rgb', class_mode='categorical' , batch_size=32)

Found 5712 validated image filenames belonging to 4 classes.
Found 655 validated image filenames belonging to 4 classes.
Found 656 validated image filenames belonging to 4 classes.


In [10]:
model = Sequential([
    Conv2D(1024, kernel_size=(3,3), activation='relu', input_shape=(224,224,3)),
    MaxPooling2D((2,2)),
    
    Conv2D(512, kernel_size=(3,3), activation='relu'),
    MaxPooling2D ((2,2)),
    
    Conv2D(256, kernel_size=(3,3), activation='relu'),
    Conv2D(128, kernel_size=(3,3), activation='relu'),
    MaxPooling2D((2,2)),
    
    Conv2D(64, kernel_size=(3,3), activation='relu'),
    MaxPooling2D((2,2)),
    
    Flatten(),
    
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    
    Dense(4, activation='softmax')

])
model.compile(optimizer=Adamax(learning_rate=0.001),loss='categorical_crossentropy',metrics=['accuracy'])

c:\Users\rohit\Desktop\Project\project\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
history = model.fit(train_gen, validation_data= valid_gen, epochs=15)

c:\Users\rohit\Desktop\Project\project\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
  2/179 ━━━━━━━━━━━━━━━━━━━━ 7:52:05 160s/step - accuracy: 0.1797 - loss: 56.9835

In [9]:
train_score = model.evaluate(train_gen)
valid_score = model.evaluate(valid_gen)
test_score = model.evaluate(test_gen)

print("Train Loss: ", train_score[0])
print("Train Accuracy: ", train_score[1])
print('-' * 20)
print("Validation Loss: ", valid_score[0])
print("Validation Accuracy: ", valid_score[1])
print('-' * 20)
print("Test Loss: ", test_score[0])
print("Test Accuracy: ", test_score[1])

179/179 ━━━━━━━━━━━━━━━━━━━━ 36s 200ms/step - accuracy: 0.9957 - loss: 0.0119
21/21 ━━━━━━━━━━━━━━━━━━━━ 4s 194ms/step - accuracy: 0.9772 - loss: 0.0914
21/21 ━━━━━━━━━━━━━━━━━━━━ 5s 230ms/step - accuracy: 0.9380 - loss: 0.2983
Train Loss:  0.012088516727089882
Train Accuracy:  0.9961484670639038
--------------------
Validation Loss:  0.09911022335290909
Validation Accuracy:  0.981679379940033
--------------------
Test Loss:  0.21721814572811127
Test Accuracy:  0.9542682766914368
